# Generation per Plant — ENTSO-E Examples

Querying actual generation at individual generator level (document type A73).

**Note:** Only generation units with ≥100 MW installed capacity are required to report.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from entsoe import Client
from entsoe._mappings import PSR_TYPES

client = Client()

## 1. All Generation Units — France (1 day)

In [ ]:
start = "2024-06-03"
end = "2024-06-04"

df = client.generation.per_plant(start, end, country="FR")
df["fuel"] = df["psr_type"].map(PSR_TYPES)
df.head(10)

In [ ]:
print(f"Columns: {df.columns.tolist()}")
print(f"Unique units: {df['unit_name'].nunique()}")
print(f"Fuel types: {df['fuel'].nunique()}")
print(f"Time range: {df['timestamp'].min()} → {df['timestamp'].max()}")

## 2. Top 10 Generators by Output

In [ ]:
totals = (
    df.groupby(["unit_name", "fuel"])["value"]
    .sum()
    .reset_index()
    .sort_values("value", ascending=False)
    .head(10)
)

fig = px.bar(
    totals, x="value", y="unit_name", color="fuel",
    orientation="h",
    title="Top 10 Generation Units by Total Output — France",
    labels={"value": "Total Generation (MWh)", "unit_name": ""},
)
fig.update_layout(yaxis={"categoryorder": "total ascending"})
fig.show()

## 3. Nuclear Plants — Hourly Profiles

In [ ]:
df_nuclear = df[df["psr_type"] == "B14"].copy()

# Pick top 5 nuclear units by total output
top_nuclear = (
    df_nuclear.groupby("unit_name")["value"].sum()
    .nlargest(5).index
)
df_top = df_nuclear[df_nuclear["unit_name"].isin(top_nuclear)]

fig = px.line(
    df_top, x="timestamp", y="value", color="unit_name",
    title="Top 5 Nuclear Units — Hourly Generation (France)",
    labels={"value": "Generation (MW)", "timestamp": "", "unit_name": "Unit"},
)
fig.show()

## 4. Gas Plants — France

In [ ]:
df_gas = df[df["psr_type"] == "B04"].copy()

fig = px.line(
    df_gas, x="timestamp", y="value", color="unit_name",
    title="Gas Plants — Individual Output (France)",
    labels={"value": "Generation (MW)", "timestamp": "", "unit_name": "Plant"},
)
fig.show()

## 5. Unit Count by Fuel Type

In [ ]:
unit_counts = (
    df.groupby("fuel")["unit_name"]
    .nunique()
    .reset_index(name="units")
    .sort_values("units", ascending=True)
)

fig = px.bar(
    unit_counts, x="units", y="fuel", orientation="h",
    title="Number of Reporting Units by Fuel Type — France",
    labels={"units": "Generation Units (≥100 MW)", "fuel": ""},
)
fig.show()